In [2]:
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
 
# Configuration de la base de données
DB_HOST = "192.168.20.131" # Adresse du serveur MySQL
DB_USER = "root"       # Nom d'utilisateur MySQL
DB_PASSWORD = "devIA25"  # Mot de passe MySQL
DB_NAME = "Projet_1_fraude"   # Nom de la base de données
 
# Connexion à la base de données
def get_db_connection():
    try:
        # Création de l'engine SQLAlchemy
        engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}")
        print("Connexion réussie à la base de données.")
        return engine
    except Exception as e:
        print(f"Erreur de connexion à la base de données : {e}")
        return None
 
# Extraction des données
def extract_data(engine):
    try:
        query = f"SELECT * FROM Transactions "  # Requête SQL pour extraire toutes les données
        data = pd.read_sql(query, engine)     # Lecture des données dans un DataFrame pandas
        print(f"Extraction réussie de {len(data)} lignes.")
        return data
    except Exception as e:
        print(f"Erreur lors de l'extraction des données : {e}")
        return None
 
# Pipeline principal
def run_pipeline():
    engine = get_db_connection()
    if engine:
        # Étape : Extraction des données
        data = extract_data(engine)
        if data is not None:
            # Affichage des 5 premières lignes
            print(data.head())
           
 
if __name__ == "__main__":
    run_pipeline()

In [3]:
DB_HOST = "192.168.20.131" # Adresse du serveur MySQL
DB_USER = "root"       # Nom d'utilisateur MySQL
DB_PASSWORD = "devIA25"  # Mot de passe MySQL
DB_NAME = "Projet_1_fraude"   # Nom de la base de données
try:
    engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}")
    query = """SELECT * FROM Transactions"""
    print("connecter !!")
except Exception as e:
    print(e)

connecter !!


In [4]:
import pandas as pd
import seaborn as sns
import numpy as np

In [5]:
data = pd.read_sql(query, engine)

In [6]:
data.head()

,transactionId,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.0,M1979787155,0.0,0.0,0
1,1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.7,M2044282225,0.0,0.0,0
2,2,1,TRANSFER,181.00,C1305486145,181.0,0.0,C553264065,0.0,0.0,1
3,3,1,CASH_OUT,181.00,C840083671,181.0,0.0,C38997010,21182.0,0.0,1
4,4,1,PAYMENT,11668.10,C2048537720,41554.0,29885.9,M1230701703,0.0,0.0,0


In [7]:
data.isna().sum()

transactionId     0
step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
dtype: int64

In [8]:
data.duplicated().sum()

np.int64(0)

In [9]:
data.groupby("isFraud")["type"].value_counts()

isFraud  type    
0        CASH_OUT    362516
         PAYMENT     329867
         CASH_IN     218938
         TRANSFER     82204
         DEBIT         6475
1        CASH_OUT      4116
         TRANSFER      4097
Name: count, dtype: int64

In [10]:
transfer_nofraud = data[(data["type"] == "TRANSFER") & (data["isFraud"] == 0)]
cashout_nofraud = data[(data["type"] == "CASH_OUT") & (data["isFraud"] == 0)]
transfer_fraud = data[(data["type"] == "TRANSFER") & (data["isFraud"] == 1)]
cashout_fraud = data[(data["type"] == "CASH_OUT") & (data["isFraud"] == 1)]

In [11]:
data  = data.drop(["nameOrig", "nameDest"], axis=1)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008213 entries, 0 to 1008212
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   transactionId   1008213 non-null  int64  
 1   step            1008213 non-null  int64  
 2   type            1008213 non-null  object 
 3   amount          1008213 non-null  float64
 4   oldbalanceOrg   1008213 non-null  float64
 5   newbalanceOrig  1008213 non-null  float64
 6   oldbalanceDest  1008213 non-null  float64
 7   newbalanceDest  1008213 non-null  float64
 8   isFraud         1008213 non-null  int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 69.2+ MB


# **Encodage**

In [13]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [14]:
colonne_categorique = data.select_dtypes(object)

In [15]:
colonne_categorique.shape

(1008213, 1)

In [16]:
encoder = OneHotEncoder(sparse_output=False)

In [17]:
colonne_encoder = encoder.fit_transform(colonne_categorique)
name_colonne_encoder = encoder.get_feature_names_out()

In [18]:
colonne_encoder.shape

(1008213, 5)

In [19]:
data_colonne_encoder = pd.DataFrame(data=colonne_encoder, columns=encoder.categories_[0])

In [20]:
new_data = pd.concat([data, data_colonne_encoder], axis=1).drop("type", axis=1)

In [22]:
new_data.to_csv("new_data.csv", index=False, sep=";")